In [1]:
from time import clock
import numpy as np
import scipy as sp
import math
import sys

def is_pos_def(V):
    '''Check to see if matrix is at worst psd
       Returns: True if pd or psd else False
    '''
    return np.all(np.linalg.eigvals(V) >= 0)

def multiplicative_nnmf(V,bas_card,p_iter=5000,random_state=0,verbose=False,adhere=False):
    '''Proposed by Lee and Seung (2000). 
       Guarantee   :  Adherence to Theorem 1 - strict adherence to Karush Kuhn Tucker condition of feasibility - (3).1 >see options
                      Preference indicated in (8)
       Returns     :  W and H
       Complexity  :  O(max(n,m)r^2) per iteration- assumed V,W,H are not sparse - as per (8)
       Mandatories :  bas_card - basis set cardinality r < min(m,n)
       Options     :  random_state - seed value - Default:0
                      iterations p_iter - update stopping criterion - Default:5000 iterations
                      adhere for guarantee 1 / autocorrect - Default:False
                      verbose for iteration information - Default:False
       Eg Usage    :  >rnd_V = np.array([np.random.uniform(1,12,23).tolist() for i in range(23)]) #need not be square!
                      >W,H = multiplicative_nnmf(rnd_V,bas_card=7,adhere=True,verbose=True)
                      >np.matmul(W,H)
       Issues      :  Lossy at high ratios
    '''
    
    if type(V) != np.ndarray:
        V = np.asarray(V)
    
    #Check adherence
    if adhere == True:
        rows = np.where(~V.any(axis=1))[0]
        cols = np.where(~V.any(axis=0))[0]
        if rows.size != 0 or cols.size != 0:
            V = np.delete(np.delete(V,rows,0),cols,1)
            print("\rViolations found and corrected...")
    
    #Read shape of input
    n,m = V.shape[0],V.shape[1]
    
    #Initialize W and H
    np.random.RandomState(random_state)
    r = bas_card
    W = np.array([np.random.uniform(0.1,1,r).tolist() for i in range(n)])
    H = np.array([np.random.uniform(0.1,1,m).tolist() for i in range(r)])
    
    #Update
    for i in range(p_iter):
        num = np.matmul(V,np.transpose(H))
        den = np.matmul(W,np.matmul(H,np.transpose(H)))
        for j in range(n):
            W[j,:] *= num[j,:]/den[j,:]
        num = np.matmul(np.transpose(W),V)
        den = np.matmul(np.matmul(np.transpose(W),W),H)
        for k in range(r):
            H[k,:] *= num[k,:]/den[k,:]
        err = round(np.sum(np.square(np.subtract(V,np.matmul(W,H))/V))/2,2)
        if verbose == True:
            sys.stdout.write("\rCurrent Normalized SSError: %.2f" % err)
            sys.stdout.flush()
        
    return W,H

def shepherds_descent_nnmf(V,bas_card,alpha=0.001,p_iter=5000,random_state=0,verbose=False):
    '''Proposed by Shepherd (2004). A projected gradient approach.
       Guarantee   :  GLB 0; convergence not guaranteed by Theorem 2 due to no explicit upper bound
                      Manner of descent updation via max(0,.) in each iteration k, implies NN W and H
       Returns     :  W and H
       Complexity  :  O(nmr) per iteration- assumed V,W,H are not sparse
       Mandatories :  bas_card - basis set cardinality/compression criteria r < min(m,n)
       Options     :  random_state - seed value - Default:0
                      step size alpha - Default:0.001
                      iterations p_iter - update stopping criterion - Default:5000 iterations
                      verbose - show iteration information - Default:False
       Eg usage    :  >rnd_V = np.array([np.random.uniform(1,23,12).tolist() for i in range(8)])
                      >W,H = shepherds_descent_nnmf(rnd_V,bas_card=5,verbose=True)
                      >np.matmul(W,H)
       Issues      :  May get stuck in local minima. Can be seen when multiplicative_nnmf
                      gives a significantly lower compression-reconstruction error.
    '''
    
    if type(V) != np.ndarray:
        V = np.asarray(V)
        
    #Read shape of input
    n,m = V.shape[0],V.shape[1]
    
    #Initialize W and H
    np.random.RandomState(random_state)
    r = bas_card
    W = np.array([np.random.uniform(0,1,r).tolist() for i in range(n)])
    H = np.array([np.random.uniform(0,1,m).tolist() for i in range(r)])
    
    #Update via unconstrained gradient descent
    for k in range(p_iter):
        grad_W = np.matmul(np.subtract(np.matmul(W,H),V),np.transpose(H))
        grad_H = np.matmul(np.transpose(W),np.subtract(np.matmul(W,H),V))
        W = np.maximum(np.subtract(W,np.multiply(alpha,grad_W)),np.zeros([n,r]))
        H = np.maximum(np.subtract(H,np.multiply(alpha,grad_H)),np.zeros([r,m]))
        err = round(np.sum(np.square(np.subtract(V,np.matmul(W,H))/V))/2,2)
        if verbose == True:
            sys.stdout.write("\rCurrent Normalized SSError: %.2f" % err)
            sys.stdout.flush()
    
    return W,H

def newton_block_nnmf(V,bas_card,p_iter=100,sub_iter=50,alpha=0.001,random_state=0,verbose=False,safe=False):
    '''Proposed by Chu (2005). *WIP* *unstable*
       Guarantee   :  GLB 0; Significantly faster convergence *unstable!*
                      Newtons method of *!optimization* root approximation to solve subproblems => approach saddle
       Returns     :  W and H
       Complexity  :  O(2nmrp) per iteration- assumed V,W,H are not sparse
       Mandatories :  bas_card - basis set cardinality/compression criteria r < min(m,n)
       Options     :  random_state - seed value - Default:0
                      sub_iter - sub-problem solving iterations for each block - 1 => regular descent
                      > 1 => approximate iterative block coordinate descent.
                      iterations p_iter - update stopping criterion - Default:100 iterations
                      verbose - show iteration information - Default:False
                      safe - safe mode - return least local minima found during descent - Default:False
       Issues      :  May not be always optimal and frequently overshoots minima - try enabling safe
                      mode to get the local minima
       Edit        :  1. Using Netwon's root approximation Order 1 instead Order 2
    '''
    
    if type(V) != np.ndarray:
        V = np.asarray(V)
        
    #Read shape of input
    n,m = V.shape[0],V.shape[1]
    
    #Initialize W and H
    np.random.RandomState(random_state)
    r = bas_card
    W = np.array([np.random.uniform(0,1,r).tolist() for i in range(n)])
    #W = np.zeros([n,r])
    H = np.array([np.random.uniform(0,1,m).tolist() for i in range(r)])
    #H = np.zeros([r,m])
    if safe == True:
        err_1 = np.inf
    
    #Update via block gradient descent
    for k in range(p_iter):
        for j in range(sub_iter):
            grad_W  = np.matmul(np.subtract(np.matmul(W,H),V),np.transpose(H))
            #grad_2W = np.matmul(H,np.transpose(H)) #square so can invert!
            grad_H  = np.matmul(np.transpose(W),np.subtract(np.matmul(W,H),V))
            #grad_2H = np.matmul(np.transpose(W),W) #square so can invert!
            #if is_pos_def(np.invert(grad_2W.astype(np.int))):
                #W = np.maximum(np.subtract(W,np.multiply(alpha,np.matmul(grad_W,np.invert(grad_2W.astype(np.int))))),np.zeros([n,r]))
            #else:
                #W = np.maximum(np.add(W,np.multiply(alpha,np.matmul(grad_W,np.invert(grad_2W.astype(np.int))))),np.zeros([n,r]))
            #print(np.invert(grad_W.astype(np.int)))
            W = np.maximum(np.add(W,np.multiply(alpha,np.invert(grad_W.astype(np.int)))),np.zeros([n,r]))
            err = round(np.sum(np.square(np.subtract(V,np.matmul(W,H))/V))/2,2)
            if verbose == True:
                sys.stdout.write("\rCurrent Normalized SSError: %.2f" % err)
                sys.stdout.flush()
            if safe == True:
                if err < err_1:
                    err_1 = err
                    W_1 = W
                    H_1 = H
        for k in range(sub_iter):
            grad_W  = np.matmul(np.subtract(np.matmul(W,H),V),np.transpose(H))
            #grad_2W = np.matmul(H,np.transpose(H)) #square so can invert!
            grad_H  = np.matmul(np.transpose(W),np.subtract(np.matmul(W,H),V))
            #grad_2H = np.matmul(np.transpose(W),W) #square so can invert!
            #if is_pos_def(np.invert(grad_2W.astype(np.int))):
                #H = np.maximum(np.subtract(H,np.multiply(alpha,np.matmul(np.invert(grad_2H.astype(np.int)),grad_H))),np.zeros([r,m]))
            #else:
                #H = np.maximum(np.add(H,np.multiply(alpha,np.matmul(np.invert(grad_2H.astype(np.int)),grad_H))),np.zeros([r,m]))
            #print(np.invert(grad_H.astype(np.int)))
            H = np.maximum(np.add(H,np.multiply(alpha,np.invert(grad_H.astype(np.int)),grad_H)),np.zeros([r,m]))
            err = round(np.sum(np.square(np.subtract(V,np.matmul(W,H))/V))/2,2)
            if verbose == True:
                sys.stdout.write("\rCurrent Normalized SSError: %.2f" % err)
                sys.stdout.flush()
            if safe == True:
                if err < err_1:
                    err_1 = err
                    W_1 = W
                    H_1 = H
    if safe == True and verbose == True:
        print("\rMinimum Normalized SSError:",err_1)
        return W_1,H_1
    elif safe == True:
        return W_1,H_1
    else:
        return W,H

def step_search(H,W,V,verbose=False):
    '''Implements the step size search to optimize cost decrease per sub-step
       Input  : H  - matrix to optimize
                W  - block held constant
                V  - input matrix
       Returns: optimized H
    '''
    
    #Initialize alpha,beta
    a = 1
    b = 0.1
    
    #Conversion buffers
    if type(W) != np.ndarray:
        W = np.array(W)
    if type(H) != np.ndarray:
        H = np.array(H)
    if type(V) != np.ndarray:
        V = np.array(V)
    
    #Calculate W'V and W'W
    WW = np.matmul(np.transpose(W),W) #O(r^2n)
    try:
        WV = np.matmul(np.transpose(W),V) #O(rmn)
    except:
        WV = np.matmul(V,np.transpose(W)) #O(rmn)
    
    path = False
    try:
        grad_H = np.subtract(np.matmul(WW,H),WV)
    except:
        grad_H = np.subtract(np.matmul(H,WW),WV)
    #Search for alpha
    #print('\nStart New')
    for j in range(20):
        #Compute new H
        H_n = np.maximum(np.subtract(H,np.multiply(a,grad_H)),np.zeros(list(H.shape)))
        diff_H = np.subtract(H_n,H)
        #Calculate condition of decrease in Cost for alpha
        d = np.subtract(H_n,H)
        gradd = np.sum(np.multiply(grad_H,d))
        dQd = np.sum(np.multiply(d,np.matmul(WW,d)))
        suff_decr = 0.99*gradd + 0.5*dQd < 0
        #Fix direction of iteration
        #print(0.99*gradd + 0.5*dQd)
        if j == 0: #Fix path to avoid convoluted exit conditions
            decr_alpha = not suff_decr
            H_p = H
        if decr_alpha: #do we go the path of decreasing learnng rate?
            if suff_decr: #do we satisfy sufficiency condition? 
                if verbose == True:
                    err = round(np.sum(np.square(np.subtract(V,np.matmul(W,H_n))/V))/2,2)
                    sys.stdout.write("\rCurrent Normalized SSError: %.2f" % err)
                    sys.stdout.flush()
                H = H_n
                break
            else:
                a = a*b
        else:
            if not suff_decr or np.allclose(H_n,H_p): #right condition is to exit when sufficient convergence cannot be achieved
                if verbose == True:
                    err = round(np.sum(np.square(np.subtract(V,np.matmul(W,H_p))/V))/2,2)
                    sys.stdout.write("\rCurrent Normalized SSError: %.2f" % err)
                    sys.stdout.flush()
                H = H_p
                break
            else:
                a = a/b
                H_p = H_n
    return H

def pgals_nnmf(V,bas_card,p_iter=50,sub_iter=100,random_state=0,verbose=False):
    '''Projected Gradient based alternating NN least squares block descent as per paper (4.1)
       Complexity  :  p_iter*(O(nmr) + sub_iter*O(tmr^2+tnr^2)); t=20 as per paper
       Guarantees  :  Asymptotic/Limiting Covergence to minima saddle as per Theorem 2 for 2 block descent
       Mandatories :  bas_card - basis set cardinality/compression criteria r < min(m,n)
       Options     :  random_state - seed value - Default:0
                      sub_iter - sub-problem solving iterations for each block - 1 => regular descent
                      > 1 => approximate iterative block coordinate descent - Default:100 sub-iterations per p_iter
                      iterations p_iter - update stopping criterion - Default:50 iterations
                      verbose - show iteration information - Default:False
       Benefits    :  self-adjusted learning at each sub-step to maximize cost reduction
    '''
    
    if type(V) != np.ndarray:
        V = np.asarray(V)
        
    #Read shape of input
    n,m = V.shape[0],V.shape[1]
    
    #Initialize W and H
    np.random.RandomState(random_state)
    r = bas_card
    W = np.array([np.random.uniform(0,1,r).tolist() for i in range(n)])
    H = np.array([np.random.uniform(0,1,m).tolist() for i in range(r)])
    
    #Update via block descent
    for k in range(p_iter):
        #print('W')
        for i in range(sub_iter):
            W = step_search(np.transpose(W),np.transpose(H),np.transpose(V),verbose=verbose)
            W = np.transpose(W)
        #print('\nH')
        for j in range(sub_iter):
            H = step_search(H,W,V,verbose=verbose)
    return W,H

In [18]:
rnd_V = np.array([np.random.uniform(0,23,12).tolist() for i in range(8)])

In [19]:
W,H = shepherds_descent_nnmf(rnd_V,bas_card=5,verbose=True)
np.matmul(W,H)

Current Normalized SSError: 12.3736

array([[13.22906688, 12.27085873, 15.49220192, 10.70200395,  6.37459773,
         8.14471545,  8.95097905, 14.24657871, 13.44272388, 18.37806974,
        14.26191189,  4.77713329],
       [10.24450048, 15.34300995,  4.44440535, 11.9650178 , 12.93573084,
         4.40548438,  2.28872522,  4.63207284, 12.36468414,  7.03169328,
         3.34459766, 22.1573894 ],
       [14.99976387,  0.        , 14.13480808, 12.50825982,  8.6738422 ,
        13.1747374 , 18.15829068,  8.05445747,  8.54261555, 19.64200748,
        10.23872389,  3.96793762],
       [ 4.22572157,  7.53180438, 17.63878835,  5.424312  , 10.27689805,
        22.48967712, 21.71946919,  4.71233045, 15.62968191,  5.63232291,
        18.14729452, 15.05688094],
       [17.95429387,  5.73519362, 10.62536297, 14.99574291,  9.21951461,
         6.09422106, 10.05131724, 10.67474605,  8.96593771, 21.48235646,
         6.72542265,  6.13896244],
       [17.57211507, 13.01868914,  7.77208434,  4.22492269, 10.54867324,
        18.05442528, 2

In [20]:
W,H = pgals_nnmf(rnd_V,bas_card=5,verbose=True)
np.matmul(W,H)

Current Normalized SSError: 12.1240

array([[13.51338466, 12.40942932, 15.31506063, 10.46871232,  6.63494181,
         8.14854964,  9.08816121, 14.29122673, 13.30303886, 18.35181441,
        14.19647963,  4.54727958],
       [10.01942755, 15.76769444,  4.66766293, 12.21555219, 13.20680441,
         4.36255067,  2.59764537,  4.71049206, 12.09683137,  6.82156002,
         3.12095183, 21.76298324],
       [15.0839374 ,  0.        , 14.36861996, 12.3424739 ,  8.73704113,
        13.30228233, 18.76181913,  7.70349095,  8.08438567, 19.55464364,
        10.45843295,  2.55545839],
       [ 3.86809623,  8.12824788, 18.66610127,  5.38627728, 10.81809788,
        22.34692923, 21.71142286,  4.15597063, 15.34290588,  5.59220287,
        17.43859202, 14.97297018],
       [18.56283458,  6.52745205,  9.59691762, 14.90758688,  9.36246308,
         5.62851372,  9.86583481, 11.22537609,  8.79370274, 21.18267102,
         6.71587078,  5.67654696],
       [17.56879726, 12.08617707,  6.23216489,  3.19449855,  9.56537501,
        18.22546004, 2

In [21]:
W,H = multiplicative_nnmf(rnd_V,bas_card=5,verbose=True)
np.matmul(W,H)

Current Normalized SSError: 12.7276

array([[1.35259043e+01, 1.24227780e+01, 1.54553647e+01, 1.04950135e+01,
        6.56964284e+00, 8.13559277e+00, 9.05254307e+00, 1.41495740e+01,
        1.32944410e+01, 1.83790671e+01, 1.42450662e+01, 4.48588527e+00],
       [9.92564518e+00, 1.59444711e+01, 4.79406637e+00, 1.21937984e+01,
        1.32860950e+01, 4.35546473e+00, 2.58932642e+00, 4.60287433e+00,
        1.20238429e+01, 7.05423275e+00, 3.06268671e+00, 2.16542024e+01],
       [1.52362571e+01, 1.80108935e-57, 1.43175972e+01, 1.22491834e+01,
        8.83071458e+00, 1.33606361e+01, 1.87898170e+01, 7.72910415e+00,
        8.04022847e+00, 1.93641549e+01, 1.06715988e+01, 2.34275888e+00],
       [4.35816410e+00, 7.90231610e+00, 1.89378888e+01, 5.45488094e+00,
        1.07586028e+01, 2.22637095e+01, 2.16889693e+01, 4.28197794e+00,
        1.53554483e+01, 5.74433804e+00, 1.70643674e+01, 1.50794103e+01],
       [1.86582292e+01, 6.68664846e+00, 9.12179128e+00, 1.46700244e+01,
        9.47704473e+00, 5.77118406e+00, 1.00103441e+01, 1.14

In [24]:
W,H = newton_block_nnmf(rnd_V,bas_card=5,verbose=True)
np.matmul(W,H)

Current Normalized SSError: 12.3796

array([[13.13965555, 12.15086957, 15.25068965, 10.4694824 ,  6.0379281 ,
         7.98694957,  8.71996052, 14.54279463, 13.35938446, 18.26918217,
        14.32908646,  4.54806396],
       [10.20320369, 15.43493503,  4.3217782 , 11.71630497, 12.89538481,
         4.09821659,  2.40682328,  4.95414942, 12.08242439,  6.6740627 ,
         3.07299638, 22.06991407],
       [14.78627599,  0.        , 13.88314603, 12.05918553,  8.42551461,
        12.79115669, 17.8000506 ,  8.93417955,  8.56257917, 19.50016051,
        10.16040397,  4.13812653],
       [ 4.20832805,  6.51315183, 17.21355707,  5.27986396, 10.21547489,
        22.35703742, 21.60054681,  4.82598493, 15.61005155,  6.13145006,
        17.93152897, 15.33116448],
       [17.37600101,  5.30661306, 11.1669528 , 15.25625494,  9.35271436,
         6.0863201 , 10.24549066, 10.38865015,  8.8376028 , 21.25519813,
         6.56827762,  6.38527717],
       [17.41663243, 13.08500763,  8.31838894,  4.09564978, 10.41954509,
        18.11219156, 2

In [25]:
rnd_V

array([[17.27304693, 12.6529049 , 14.39565645, 12.65550079,  2.50644045,
         9.11051508,  8.83182354, 11.46981118, 16.65025458, 15.79717371,
        14.69734402,  3.42358053],
       [12.10508645, 17.17633431,  4.60849711, 10.61172763, 13.14780492,
         4.71704934,  2.46736094,  2.46237329, 11.76201642,  7.14814846,
         3.42479451, 21.22538504],
       [18.56390363,  0.09601965, 13.92257974,  8.94437808,  8.32120717,
        13.964711  , 18.17313256,  2.78208614,  7.80995048, 21.13515186,
        11.63096583,  1.92014144],
       [ 1.42920956,  7.62993003, 19.4030864 ,  6.57115518, 12.10970014,
        21.9039962 , 22.39277788,  7.21087625, 14.41616965,  4.75713064,
        16.26410043, 15.04213776],
       [12.18051747,  2.08269448, 10.13398037, 18.39551909, 10.38169984,
         4.2460275 ,  9.74409215, 17.53466117,  9.21707785, 21.54462749,
         6.00212461,  7.81936373],
       [16.72293514,  9.20176112,  3.47659751,  4.1524457 ,  6.91350557,
        18.44415692, 1